# A small python selenium protocol to scrap article titles per year from the journal "Cell" through pubmed

In [1]:
### import libraries for selenium

import time
import lxml.html as html
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC, wait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from selenium.webdriver.firefox.options import Options

In [2]:
### define options for webdriver

options = Options()
options.add_argument("--ignore-certificate-errors")
options.add_argument("--incognito")
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

In [3]:
### scrap first shoe page for single shoe urls

from bs4 import BeautifulSoup as bs
import pandas as pd

url1 = 'https://pubmed.ncbi.nlm.nih.gov/?term=%22Cell%22%5BJournal%5D&sort=pubdate&size=200'
driver = webdriver.Firefox(options=options)
driver.get(url1)
page_source = driver.page_source
soup = bs(page_source,features="html.parser")

article_css = '.search-results-chunk'

articles = soup.select(article_css)
all_results = [r.text for r in articles]
result = all_results[0].split("Share\n    \n\n\n\n\n\n\n              \n                ")
result2 = [r.split("\n")[0] for r in result[1:]]
result3 = [r.split(".\nCell. ")[1] for r in result[1:]]
result4 = [r[:4] for r in result3]
TitlesYears = pd.DataFrame({'Titles': result2, 'Years': result4})
driver.close()

In [4]:
print(TitlesYears)

                                                Titles Years
0    Tumor-associated macrophages trigger MAIT cell...  2023
1    A conserved family of immune effectors cleaves...  2023
2    An ethical framework for human embryology with...  2023
3    Insights and strategies for improving equity i...  2023
4    RNA "COURIERs": Enabling synthetic cell-to-cel...  2023
..                                                 ...   ...
195  Negative allosteric modulation of the glucagon...  2023
196  A neutrophil response linked to tumor control ...  2023
197  T cell immunotherapies engage neutrophils to e...  2023
198  Cytotoxic CD4+ T cells eliminate senescent cel...  2023
199  Anteromedial thalamus gates the selection and ...  2023

[200 rows x 2 columns]


In [6]:
import time
from tqdm import tqdm

url1 = 'https://pubmed.ncbi.nlm.nih.gov/?term=%22Cell%22%5BJournal%5D&sort=pubdate&size=200'
driver = webdriver.Firefox(options=options)
driver.get(url1)
articles_list = []

for i in tqdm(list(range(2, 50))):
# Click on United States button to enter the desired country on the pop-up
    next_button = driver.find_element(By.XPATH, '/html/body/main/div[9]/div[2]/div[2]/div[2]/button[3]')
    next_button.click()
    time.sleep(2)
    page_source = driver.page_source
    soup = bs(page_source,features="html.parser")
    
    article_css = '.search-results-chunk'
    
    articles = soup.select(article_css)
    all_results = [r.text for r in articles]
    result = all_results[0].split("Share\n    \n\n\n\n\n\n\n              \n                ")
    result2 = [r.split("\n")[0] for r in result[1:]]
    result3 = [r.split(".\nCell. ")[1] for r in result[1:]]
    result4 = [r[:4] for r in result3]
    articlesyear = pd.DataFrame({'Titles': result2, 'Years': result4})
    articles_list.append(articlesyear)
driver.close()

100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [07:08<00:00,  8.92s/it]


In [8]:
print(articles_list[46])

                                                Titles Years
0    Drosophila IKK-related kinase regulates nonapo...  2006
1    New evidence supporting megakaryocyte-erythroc...  2006
2    Interchromosomal interactions and olfactory re...  2006
3    In vivo two-photon imaging reveals a role of a...  2006
4    FOXP3 controls regulatory T cell function thro...  2006
..                                                 ...   ...
195  Cellular conference call: external feedback af...  2006
196  The KEOPS complex: a rosetta stone for telomer...  2006
197  Increasingly irritable and close to tears: TRP...  2006
198  Connecting cancer to the asymmetric division o...  2006
199  LRP6 holds the key to the entry of anthrax toxin.  2006

[200 rows x 2 columns]


In [11]:
### combine to one pandas frame

for elem in articles_list:
    TitlesYears = pd.concat([TitlesYears, elem], ignore_index=True)
    
print(TitlesYears)

                                                  Titles Years
0      Tumor-associated macrophages trigger MAIT cell...  2023
1      A conserved family of immune effectors cleaves...  2023
2      An ethical framework for human embryology with...  2023
3      Insights and strategies for improving equity i...  2023
4      RNA "COURIERs": Enabling synthetic cell-to-cel...  2023
...                                                  ...   ...
28995  A slowed classical pathway rather than kiss-an...  2005
28996  Exploration of the function and organization o...  2005
28997  Divergence of melanocortin pathways in the con...  2005
28998  A Hox regulatory network establishes motor neu...  2005
28999  Calibrated measurement of gating-charge argini...  2005

[29000 rows x 2 columns]


In [12]:
### save results

TitlesYears.to_csv('CellArticleTitles.csv')